# Capstone project: Weather classification project  

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.6.0'

In [6]:
train_datagen= tf.keras.preprocessing.image.ImageDataGenerator(
    # DEFAULT PARAMS 
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0,
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0.0,
    channel_shift_range=0.0,
    fill_mode='nearest',
    cval=0.0,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=None,
    data_format=None,
    dtype=None,

    # PARAMS TO EXPLORE
    validation_split=0.3,
    zoom_range=0.0,
    preprocessing_function=None
)

In [8]:
train_generator = train_datagen.flow_from_directory(
    r'C:\Users\User\@Code-ML\Zoom-camp Capstone Project\Data\dataset',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    r'C:\Users\User\@Code-ML\Zoom-camp Capstone Project\Data\dataset', # same directory as training data
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation') # set as validation data

Found 3538 images belonging to 8 classes.
Found 1512 images belonging to 8 classes.


In [9]:
train_generator.class_indices

{'fogsmog': 0,
 'hail': 1,
 'lightning': 2,
 'rain': 3,
 'rainbow': 4,
 'rime': 5,
 'sandstorm': 6,
 'snow': 7}

In [11]:
X, y = next(train_generator)

In [14]:
(X[:5]).shape

(5, 150, 150, 3)